In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import sys
sys.path.append('..')
import misc.autointerpolation

# Reload misc module after editing
import importlib
importlib.reload(misc.autointerpolation)
importlib.reload(misc)

<module 'misc' (namespace)>

# 1. Load and Preprocess Data <a class="anchor" id="load_data"></a>

In [2]:
# path = "E:/Projects/SEACAR_WQ_Pilot/"
path = "../../"
gis_path = path+'GIS_data/'

# path = "D:/Water_Quailty/"
# gis_path = path+'GIS_data/'

Combine and preprocess discrete and continuous data.

In [3]:
# dfDis, dfCon = misc.preprocess.preprocess(path + r"OEAT_Discrete-2023-Jan-23.csv", 
#                     path + r"OEAT_Continuous_1-2023-Jan-23.csv",
#                     path + r"OEAT_Continuous_2-2023-Jan-23.csv")

# dfAll = pd.concat([dfDis,dfCon]).drop(columns = ['Hour'])

# dfAll.to_csv(path + "all_0214.csv")

In [4]:
# Reload modules in external .py files after editing.
dfAll = pd.read_csv(path + "all_0214.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1','RowID','ValueQualifier'])
# Convert SampleDate froms str to date
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate']).dt.date

C:\Users\yi\AppData\Local\Temp\ipykernel_4792\3834789934.py:2: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  dfAll = pd.read_csv(path + "all_0214.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1','RowID','ValueQualifier'])


In [5]:
col_ls = ['RowID','ParameterName','ParameterUnits','ProgramLocationID','ActivityType','ManagedAreaName',
                   'SampleDate','Year','Month','ResultValue','ValueQualifier','Latitude_DD','Longitude_DD']
para_ls = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
para_ls_ab = ["S","TN","DO","T","SD"]
# Convert full MA names to short names
dictArea    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'GTM Reserve','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}

# Convert full MA names to MA name in ORCP_Managed_Areas_Oct2021
dictArea2    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Gasparilla Sound-Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend Seagrasses',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'Guana Tolomato Matanzas NERR','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island Sound'}

# Convert full MA names to abbreviations
dictArea3    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'ch','Big Bend Seagrasses Aquatic Preserve':'bb',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'gtm','Estero Bay Aquatic Preserve':'eb',
                'Biscayne Bay Aquatic Preserve':'bbay','Matlacha Pass Aquatic Preserve':'mp',
                'Lemon Bay Aquatic Preserve':'lb','Cape Haze Aquatic Preserve':'ch','Pine Island Sound Aquatic Preserve':'pi'}

dictPara = {"Salinity":'S','Total Nitrogen':'TN','Dissolved Oxygen':'DO','Turbidity':'T','Secchi Depth':'SD'}
dictUnits   = {"Salinity":"ppt","Total Nitrogen": "mg/L","Dissolved Oxygen": "mg/L","Turbidity": "NTU", "Secchi Depth": "m"}
listArea    = dfAll["ManagedAreaName"].unique()
listPara    = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
SpatialRef = '3086'

# 2. Run spatial interpolation through seasonality table <a class="anchor" id="combine"></a>

Delete incorrect seasons

In [6]:
dfSeason = pd.read_csv(path + "OEATUSF_Geospatial_TempSeasons_update_0611_added.csv").drop(columns = "Unnamed: 0")
dfSeason = dfSeason.dropna(subset=["s_start","s_end"]).reset_index(drop=True)

dfSeason['s_start'] = pd.to_datetime(dfSeason['s_start'])
dfSeason["s_end"] = pd.to_datetime(dfSeason["s_end"])
dfSeason["date_range"] = dfSeason["s_end"] - dfSeason["s_start"]
dfSeason["date_range"] = dfSeason.apply(lambda x: int(str(x["date_range"]).split(" ")[0]),axis=1)
dfSeason = dfSeason[dfSeason["date_range"]>0]
dfSeason.head()

,Unnamed: 0.1,param,ma,st_Year,season,med_seamo_ma,s_start,s_end,date_range,covariates,Processed,Total Nitrogen,Dissolved Oxygen,Salinity,Secchi Depth,Turbidity
0,377,temp,Big Bend Seagrasses Aquatic Preserve,2018,Fall,31-Aug,2018-08-31,2019-01-14,136,bathymetry+LDI,1,NaN,NaN,NaN,NaN,NaN
1,378,temp,Big Bend Seagrasses Aquatic Preserve,2018,Summer,14-Jun,2018-06-14,2018-08-30,77,bathymetry+LDI,1,NaN,NaN,NaN,NaN,NaN
2,379,temp,Big Bend Seagrasses Aquatic Preserve,2018,Spring,18-Feb,2018-02-18,2018-06-13,115,bathymetry+LDI,1,NaN,NaN,NaN,NaN,NaN
3,380,temp,Big Bend Seagrasses Aquatic Preserve,2018,Winter,15-Jan,2018-01-15,2018-02-17,33,bathymetry+LDI,1,NaN,NaN,NaN,NaN,NaN
4,381,temp,Big Bend Seagrasses Aquatic Preserve,2017,Fall,31-Aug,2017-08-31,2018-01-14,136,bathymetry+LDI,1,NaN,NaN,NaN,NaN,NaN


In [7]:
# Print all managed areas
display(dfSeason['ma'].unique())

# Create empty columns to store paths of exported files
# data = dfSeason.assign(**{'Total Nitrogen': np.nan, 'Dissolved Oxygen': np.nan, 'Salinity': np.nan, 'Secchi Depth': np.nan, 'Turbidity': np.nan})

array(['Big Bend Seagrasses Aquatic Preserve'], dtype=object)

Run through all seasons

In [8]:
for i in dfSeason.index:
#for i in range(177, 178):
    # if dfSeason.iloc[i]["Processed"] == 1: continue
    for para in listPara[0:5]:
        print("Now interpolating: Row {}; Area {}; Year {}; Season {}; para {}".format(i,dictArea3[dfSeason.iloc[i]["ma"]], dfSeason.iloc[i]["st_Year"], dfSeason.iloc[i]["season"],para))
        out_raster,out_ga_layer,ga_to_raster = misc.autointerpolation.interpolation_auto(method = "rk",
                                                                                         gis_path  = gis_path,
                                                                                         dataframe = dfAll,
                                                                                         managed_area = dfSeason.iloc[i]["ma"],
                                                                                         Year = dfSeason.iloc[i]["st_Year"],
                                                                                         Season = dfSeason.iloc[i]["season"],
                                                                                         start_date = dfSeason.iloc[i]["s_start"],
                                                                                         end_date = dfSeason.iloc[i]["s_end"],
                                                                                         parameter = para,
                                                                                         covariates = dfSeason.iloc[i]["covariates"].split("+"))
#                                                                                          out_raster = name,
#                                                                                          out_ga_layer = name,
#                                                                                          predict_std_err = name)
        dfSeason.loc[i, para] = out_raster
        dfSeason.to_csv(path + 'output.csv')

Now interpolating: Row 0; Area bb; Year 2018; Season Fall; para Salinity


C:\projects\SEACAR_WQ_Pilot\git\Spatial_Interpolation\..\misc\autointerpolation.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(input_pt,driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)


Salinity in 2018 Fall caused an error:
Invalid version: '3.4.0e'
Now interpolating: Row 0; Area bb; Year 2018; Season Fall; para Total Nitrogen


C:\projects\SEACAR_WQ_Pilot\git\Spatial_Interpolation\..\misc\autointerpolation.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(input_pt,driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)


Total Nitrogen in 2018 Fall caused an error:
Invalid version: '3.4.0e'
Now interpolating: Row 0; Area bb; Year 2018; Season Fall; para Dissolved Oxygen


...

C:\projects\SEACAR_WQ_Pilot\git\Spatial_Interpolation\..\misc\autointerpolation.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(input_pt,driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)


Dissolved Oxygen in 2016 Winter caused an error:
Invalid version: '3.4.0e'
Now interpolating: Row 11; Area bb; Year 2016; Season Winter; para Turbidity


C:\projects\SEACAR_WQ_Pilot\git\Spatial_Interpolation\..\misc\autointerpolation.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(input_pt,driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)


Turbidity in 2016 Winter caused an error:
Invalid version: '3.4.0e'
Now interpolating: Row 11; Area bb; Year 2016; Season Winter; para Secchi Depth
Secchi Depth in 2016 Winter caused an error:
Invalid version: '3.4.0e'


C:\projects\SEACAR_WQ_Pilot\git\Spatial_Interpolation\..\misc\autointerpolation.py:173: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(input_pt,driver='ESRI Shapefile',crs="EPSG:"+SpatialRef)
